In [1]:
import pandas as pd
import numpy as np
import tqdm
import torch
import math

# Define Model

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import EdgeConv, global_mean_pool
import torch
import torch_geometric.transforms as T
from torch_geometric.nn import EdgeConv, global_mean_pool
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_scatter import scatter_mean
from torch_geometric.nn import MetaLayer

# GNN AE using EdgeConv (mean aggregation graph operation) and embedding. GAE model with embedding.
class EdgeNetEmbed(nn.Module):
    def __init__(self, input_dim=4, big_dim=32, hidden_dim=2, aggr='mean'):
        super(EdgeNetEmbed, self).__init__()
        self.embed_nn = nn.Sequential(nn.Linear(input_dim, big_dim),
                                      nn.ReLU(),
                                      nn.Linear(big_dim, big_dim),
                                      nn.ReLU(),
                                      nn.Linear(big_dim, big_dim),
                                      nn.ReLU()
        )                                
        encoder_nn = nn.Sequential(nn.Linear(2*(big_dim), big_dim),
                                   nn.ReLU(),
                                   nn.Linear(big_dim, big_dim),
                                   nn.ReLU(),
                                   nn.Linear(big_dim, hidden_dim),
                                   nn.ReLU(),
        )        
        decoder_nn = nn.Sequential(nn.Linear(2*(hidden_dim), big_dim),
                                   nn.ReLU(),
                                   nn.Linear(big_dim, big_dim),
                                   nn.ReLU(),
                                   nn.Linear(big_dim, big_dim),
                                   nn.ReLU()
                                   
        )
        self.deembed_nn = nn.Sequential(nn.Linear(big_dim, big_dim),
                                        nn.ReLU(),
                                        nn.Linear(big_dim, big_dim),
                                        nn.ReLU(),
                                        nn.Linear(big_dim, input_dim)
        )                                
        
        self.batchnorm = nn.BatchNorm1d(input_dim)

        self.encoder = EdgeConv(nn=encoder_nn,aggr=aggr)
        self.decoder = EdgeConv(nn=decoder_nn,aggr=aggr)

    def forward(self, data):
        data.x = self.batchnorm(data.x)
        data.x = self.embed_nn(data.x)
        data.x = self.encoder(data.x,data.edge_index)
        data.x = self.decoder(data.x,data.edge_index)
        data.x = self.deembed_nn(data.x)
        return data.x

In [3]:
from torch_geometric.data import Data, DataListLoader, Batch
from torch.utils.data import random_split
import os.path as osp
import matplotlib.pyplot as plt
from graph_data import GraphDataset

gdata = GraphDataset(root='/anomalyvol/data/gnn_node_global_merge', bb=0)

input_dim = 4
big_dim = 32
hidden_dim = 2
fulllen = len(gdata)
tv_frac = 0.10
tv_num = math.ceil(fulllen*tv_frac)
batch_size = 2
n_epochs = 100
lr = 0.001
patience = 10
device = 'cuda:0'
model_fname = 'EdgeConv_Embed'

model = EdgeNetEmbed(input_dim=input_dim, big_dim=big_dim, hidden_dim=hidden_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [4]:
def collate(items):
    l = sum(items, [])
    return Batch.from_data_list(l)

In [5]:
torch.manual_seed(0)
train_dataset, valid_dataset, test_dataset = random_split(gdata, [fulllen-2*tv_num,tv_num,tv_num])

train_loader = DataListLoader(train_dataset, batch_size=batch_size, pin_memory=True, shuffle=True)
train_loader.collate_fn = collate
valid_loader = DataListLoader(valid_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
valid_loader.collate_fn = collate
test_loader = DataListLoader(test_dataset, batch_size=batch_size, pin_memory=True, shuffle=False)
test_loader.collate_fn = collate

train_samples = len(train_dataset)
valid_samples = len(valid_dataset)
test_samples = len(test_dataset)

print(train_samples)
print(valid_samples)
print(test_samples)

8000
1000
1000


In [6]:
@torch.no_grad()
def test(model,loader,total,batch_size):
    model.eval()
    
    mse = nn.MSELoss(reduction='mean')

    sum_loss = 0.
    t = tqdm.tqdm(enumerate(loader),total=total/batch_size)
    for i,data in t:
        data = data.to(device)
        y = data.x # the model will overwrite data.x, so save a copy
        batch_output = model(data)
        batch_loss_item = mse(batch_output, y).item()
        sum_loss += batch_loss_item
        t.set_description("loss = %.5f" % (batch_loss_item))
        t.refresh() # to show immediately the update

    return sum_loss/(i+1)

def train(model, optimizer, loader, total, batch_size):
    model.train()
    
    mse = nn.MSELoss(reduction='mean')

    sum_loss = 0.
    t = tqdm.tqdm(enumerate(loader),total=total/batch_size)
    for i,data in t:
        data = data.to(device)
        y = data.x # the model will overwrite data.x, so save a copy
        optimizer.zero_grad()
        batch_output = model(data)
        batch_loss = mse(batch_output, y)
        batch_loss.backward()
        batch_loss_item = batch_loss.item()
        t.set_description("loss = %.5f" % batch_loss_item)
        t.refresh() # to show immediately the update
        sum_loss += batch_loss_item
        optimizer.step()
    
    return sum_loss/(i+1)

In [7]:
modpath = osp.join('/anomalyvol/models/',model_fname+'.best.pth')
try:
    model.load_state_dict(torch.load(modpath))
except:
    pass

# Train

In [ ]:
stale_epochs = 0
best_valid_loss = 99999
for epoch in range(0, n_epochs):
    loss = train(model, optimizer, train_loader, train_samples, batch_size)
    valid_loss = test(model, valid_loader, valid_samples, batch_size)
    print('Epoch: {:02d}, Training Loss:   {:.4f}'.format(epoch, loss))
    print('               Validation Loss: {:.4f}'.format(valid_loss))

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        modpath = osp.join('/anomalyvol/models/gnn/',model_fname+'.best.pth')
        print('New best model saved to:',modpath)
        torch.save(model.state_dict(),modpath)
        stale_epochs = 0
    else:
        print('Stale epoch')
        stale_epochs += 1
    if stale_epochs >= patience:
        print('Early stopping after %i stale epochs'%patience)
        break

loss = 2112.64893:  68%|██████▊   | 2707/4000.0 [1:41:29<41:49,  1.94s/it]  

# Visualize

In [ ]:
model.load_state_dict(torch.load(modpath))
input_x = []
output_x = []

t = tqdm.tqdm(enumerate(test_loader),total=test_samples/batch_size)
for i, data in t:
    data.to(device)
    input_x.append(data.x.cpu().numpy())
    output_x.append(model(data).cpu().detach().numpy())
    del data
    torch.cuda.empty_cache()

In [ ]:
def in_out_diff_append(diff, output, inputs, i, ft_idx): # helper for appending 3 lists
    diff.append(((output_x[i][:,ft_idx]-input_x[i][:,ft_idx])/input_x[i][:,ft_idx]).flatten())
    output.append(output_x[i][:,ft_idx].flatten())
    inputs.append(input_x[i][:,ft_idx].flatten())

def in_out_diff_concat(diff, output, inputs):
    diff = np.concatenate(diff)
    output = np.concatenate(output)
    inputs = np.concatenate(inputs)
    return [diff, output, inputs]

def make_hists(diff, output, inputs, bin1, feat):
    plt.figure(figsize=(6,4.4))
    plt.hist(inputs, bins=bin1,alpha=0.5, label='input')
    plt.hist(output, bins=bin1,alpha=0.5, label='output')
    plt.legend()
    plt.xlabel(feat, fontsize=14)
    plt.ylabel('Particles', fontsize=14)
    plt.show()

    plt.figure()
    plt.hist(diff, bins=np.linspace(-5, 5, 101))
    plt.show()

In [ ]:
diff_px = []
output_px = []
input_px = []
diff_py = []
output_py = []
input_py = []
diff_pz = []
output_pz = []
input_pz = []
diff_e = []
output_e = []
input_e = []

# get output in readable format
for i in range(len(input_x)):
    # px
    in_out_diff_append(diff_px, output_px, input_px, i, 0)
    in_out_diff_append(diff_py, output_py, input_py, i, 1)
    in_out_diff_append(diff_pz, output_pz, input_pz, i, 2)
    in_out_diff_append(diff_e, output_e, input_e, i, 3)

# remove extra brackets
diff_px, output_px, input_px = in_out_diff_concat(diff_px, output_px, input_px)
diff_py, output_py, input_py = in_out_diff_concat(diff_py, output_py, input_py)
diff_pz, output_pz, input_pz = in_out_diff_concat(diff_pz, output_pz, input_pz)
diff_e, output_e, input_e = in_out_diff_concat(diff_e, output_e, input_e)

# make plots
feat = '$p_x~[GeV]$'
bins = np.linspace(-20, 20, 101)
make_hists(diff_px, output_px, input_px, bins, feat)

feat = '$p_y~[GeV]$'
make_hists(diff_py, output_py, input_py, bins, feat)

feat = '$p_z~[GeV]$'
make_hists(diff_pz, output_pz, input_pz, bins, feat)

feat = '$E~[GeV]$'
bins = np.linspace(-5, 35, 101)
make_hists(diff_e, output_e, input_e, bins, feat)

# Bump Hunt

In [ ]:
# m_12 = sqrt ( (E_1 + E_2)^2 - (p_x1 + p_x2)^2 - (p_y1 + p_y2)^2 - (p_z1 + p_z2)^2 )
def invariant_mass(jet1, jet2):
    return math.sqrt((jet1.e + jet2.e)**2 - (jet1.px + jet2.px)**2 - (jet1.py + jet2.py)**2 - (jet1.pz + jet2.pz)**2)

In [ ]:
# generates bump hunt histograms for given dataset
def bump_hunt(dataset, cut=0.995):
    
    # get losses and other useful jet info
    losses = []
    event_idx = []
    masses = []
    px = []
    py = []
    pz = []
    e = []
    model.eval()
    mse = nn.MSELoss(reduction='mean')
    t = tqdm.tqdm(enumerate(dataset))
    for i, data in t:
        event_idx.append(data[1])
        masses.append(data[2])
        px.append(data[3])
        py.append(data[4])
        pz.append(data[5])
        e.append(data[6])
        try:
            with torch.no_grad():
                data = data[0].to(device)
                batch_output = model(data)
                batch_loss_item = mse(batch_output, data.y).item()
                losses.append(batch_loss_item)
        except:
            losses.append(0) # model can't evaluate tiny jet

    # make a cut on the loss
    mse_thresh = np.quantile(losses, cut)
    outliers = losses > mse_thresh
    
    # create dataframe; jet per row
    d = {'event':event_idx, 'mass':masses, 'outlier':outliers, 'px':px, 'py':py, 'pz':pz, 'e':e}
    df = pd.DataFrame(d)

    # calculate invariant dijet mass using leading 2 jets of all events
    dijet_mass = []
    outlier_mass = []
    for event, group in df.groupby(['event']):
        try:
            im = invariant_mass(group.iloc[0], group.iloc[1])
            dijet_mass.append(im)
            
            # if one of leading 2 jets is outlier then get dijet invariant mass
            if group.iloc[0].outlier == True or group.iloc[1].outlier == True:
                outlier_mass.append(im)
        except:
            pass # when event only has 1 jet

    # plot mjj bump histograms
    bins = np.linspace(1000, 6000, 51)
    weights = np.ones_like(outlier_mass) / len(outlier_mass)
    plt.hist(np.array(outlier_mass), alpha = 0.5, bins=bins, weights=weights)
    weights = np.ones_like(dijet_mass) / len(dijet_mass)
    plt.hist(np.array(dijet_mass), alpha = 0.5, bins=bins, weights=weights)
    plt.show()

In [ ]:
bb1_data = GraphDataset(root='/anomalyvol/data/gnn_geom/full', bb=1)
bump_hunt(bb1_data)

In [ ]:
bb2_data = GraphDataset(root='/anomalyvol/data/gnn_geom/full', bb=2)
bump_hunt(bb2_data)

In [ ]:
bb3_data = GraphDataset(root='/anomalyvol/data/gnn_geom/full', bb=3)
bump_hunt(bb3_data)